In [44]:
import pandas as pd

df = pd.read_csv("liquidity_zscores_labeled.csv")

# Show column names clearly
for col in df.columns:
    print(f"'{col}'")

'Date'
'M2 MoM %'
'NFCI (Financial Conditions Index)'
'10Y Treasury Yield MoM %'
'3-Month T-Bill Rate (%)'
'10Y 3M Treasury Spread'
'TED Spread (bps)'
'M2 MoM %_Label'
'NFCI (Financial Conditions Index)_Label'
'10Y Treasury Yield MoM %_Label'
'3-Month T-Bill Rate (%)_Label'
'10Y 3M Treasury Spread_Label'
'TED Spread (bps)_Label'


In [46]:
df.columns = df.columns.str.strip()

df.rename(columns={
    'M2 MoM %': 'M2_MoM_Z',
    'NFCI (Financial Conditions Index)': 'NFCI_Z',
    '10Y 3M Treasury Spread': 'YieldCurve_10Y3M_Z',
    'TED Spread (bps)': 'TED_Spread_Z'
}, inplace=True)


In [47]:
def normalized_contribution(z, weight, cap=2.0):
    z = max(min(z, cap), -cap)        # Cap between -2 and +2
    scaled = (z + cap) / (2 * cap)    # Normalize to 0–1 scale
    return scaled * weight


In [48]:
df['MoneySupply_SubTheme_Score'] = df['M2_MoM_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100

df['Credit_SubTheme_Score'] = df['TED_Spread_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100

df['Financial_Tightness_SubTheme_Score'] = df['NFCI_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100

df['YieldCurve_SubTheme_Score'] = df['YieldCurve_10Y3M_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100


In [49]:
weights_liquidity = {
    'MoneySupply_SubTheme_Score': 0.3,
    'Credit_SubTheme_Score': 0.25,
    'Financial_Tightness_SubTheme_Score': 0.25,
    'YieldCurve_SubTheme_Score': 0.2
}

df['Liquidity_Theme_Score'] = sum(
    df[col] * weight for col, weight in weights_liquidity.items()
).round(2)


In [50]:
print(df[['Date', 'Liquidity_Theme_Score']].head())
next 

         Date  Liquidity_Theme_Score
0  1986-02-01                  59.55
1  1986-03-01                  54.20
2  1986-04-01                  58.68
3  1986-05-01                  54.39
4  1986-06-01                  55.42


<function next>

In [51]:
df[['Date',
    'MoneySupply_SubTheme_Score',
    'Credit_SubTheme_Score',
    'Financial_Tightness_SubTheme_Score',
    'YieldCurve_SubTheme_Score',
    'Liquidity_Theme_Score',
]].to_csv("liquidity_theme_output.csv", index=False)